In [2]:
import pandas as pd 
import numpy as np 
import requests
import csv 
import re
from bs4 import BeautifulSoup
from selenium import webdriver

In [150]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [4]:
def open_browser():
    driver = webdriver.Chrome("K:\code\work_dev/chromedriver")
    driver.get("https://property.sulekha.com/apartments-flats-for-rent/pune#")
    driver.find_element_by_id('bt_gerar_cpf').click()
    text_field = driver.find_element_by_id('texto_cpf')
    text = wait(driver, 10).until(lambda driver: not text_field.text == 'Gerando...' and text_field.text)
    return text

print(open_browser())
open_browser()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bt_gerar_cpf"]"}
  (Session info: chrome=89.0.4389.114)


In [36]:
def title_link(s):
    tit=[]
    link=[]
    sta=s.find_all('a',attrs={'class':"bizlinkurl"}) 
    for k in sta:
        tit.append(k.text)
        link.append(k['href'])
    d={
        'title_link':tit,
        'links':link
    }
    df=pd.DataFrame(d)
    return df

In [37]:
def address(s):
    address=[]
    sta=s.find_all('div',attrs={'class':"locality"}) 
    for a in sta:
        address.append(a.text)
    return address


 

In [38]:
def location(s):
    location=[]
    sta=s.find_all('a',attrs={'class':"view-on-map"})    
    for a in sta:
        location.append(a['data-latlong'])
    return location

In [49]:
def prices(s):
    prices=[]
    sta=s.find_all('span',attrs={'class':"deposit"})    
    for i in sta:
        prices.append(i.text)
    """    sta=s.find_all('span',attrs={'class':"Deposit"})    
    for i in sta:
        prices.append(i.text)"""
    return prices

In [40]:
def financial(s):
    financial=[]
    sta=s.find_all('span',attrs={'class':"price primary"})    
    for i in sta:
        financial.append(i.text)
    return financial

In [53]:
def amenities(s):
    ammenities=[]
    sta=s.find_all('div',attrs={'class':"amenities flat"})   
    for i in sta:
        ammenities.append(i.text)
    return ammenities

In [42]:
def spec_flat(s):
    spec_flat=[]
    sta=s.find_all('div',attrs={'class':"spec flat"})   
    for i in sta:
        spec_flat.append(i.text)
    return spec_flat

In [43]:
def spec_flat_necce(s):
    spec_flat_necce=[]
    sta=s.find_all('div',attrs={'class':"spec icon primary"})   
    for i in sta:
        spec_flat_necce.append(i.text)
    return spec_flat_necce

In [44]:
def dates_bro(s):
    dates_bro=[]
    sta=s.find_all('div',attrs={'class':"posted"})   
    for i in sta:
        dates_bro.append(i.text)
    return dates_bro

In [45]:
def the_text(s):
    the_text=[]
    sta=s.find_all('span',attrs={'class':"listing-desc view-less"})   
    for i in sta:
        the_text.append(i.text)

In [46]:
url="https://property.sulekha.com/apartments-flats-for-rent/pune#"
j=requests.get(url)
k=j.text
s=BeautifulSoup(k,'lxml')

In [145]:
def the_filter(df):
    the_liost=df['financial']
    prices=[]
    deposit=[]
    for l in the_liost:
        k=l.split(' ')
        prices.append(k[3])
        deposit.append(k[-4])
    df['prices']=prices
    df['deposit']=deposit
    k=df['dates_bro']
    the_posted_dates=[]
    details_of=[]
    for i in k :
        i=i.strip()
        i=i.split(" ")
        the_posted_dates.append(i[-3:])
        details_of.append(' '.join(i[0:-3]))
    df['the_posted_dates']=the_posted_dates
    df['details']=details_of
    df_new=df.drop(['financial',	'dates_bro'], axis = 1)
    df_new=df_new.drop(['amenities'],axis=1)
    k=len(df['the_posted_dates'])
    the_date=[]
    for i in range(0,k):
        j=df['the_posted_dates'][i]
        the_date.append(" ".join(j))
    df_new['the_posted_dates']=pd.to_datetime(the_date)
    return df_new

In [146]:
df=title_link(s)
df['adress']=address(s)
#df['prices']=prices(s)
df['financial']=financial(s)
df['dates_bro']=dates_bro(s)
df['spec_flat']=spec_flat(s)
df['spec_flat_necce']=spec_flat_necce(s)
df['amenities']=amenities(s)
df['location']=location(s)
df=the_filter(df)

In [147]:
df

,title_link,links,adress,spec_flat,spec_flat_necce,location,prices,deposit,the_posted_dates,details
0,2 BHK Residential Apartment for Rent at kumar ...,/apartments-flats-for-rent/1150-sqft-2-bhk-fla...,"Hadapsar, Pune View Map",1 Car Parking 4th (of 11 Floors) UnFurnished F...,Corporation water Drainage Facility 24*7 Power...,"18.515806,73.927164",for,"45,000",2021-03-01,Building: kumar paradise Broker: Sejal Prop...
1,2 BHK Residential Apartment for Rent at La Glo...,/apartments-flats-for-rent/1050-sqft-2-bhk-apa...,"Kalyani Nagar, Pune Landmark: Near Nyati m...",2 Car Parking 6th (of 7 Floors) Fully Furnishe...,Drainage Facility Lifts Security +22 Amenities,"0,0",26,"80,000",2020-12-11,Building: La Gloriosa Broker: Mountain Gate...
2,1 RK Residential Apartment for Rent in Baner Gaon,/apartments-flats-for-rent/250-sqft-1-rk-flat-...,"Baner Gaon, Pune Landmark: Orchid school ...",1 Car Parking 2nd (of 3 Floors) Semi Furnished...,Feng Shui / Vastu,"0,0","7,000","20,000",2021-03-08,Broker: Life Real Estates
3,1 RK Residential Apartment for Rent in Pavana ...,/apartments-flats-for-rent/300-sqft-1-rk-flat-...,"Pavana Nagar, Pune View Map",No Car Parking 1st (of 1 Floors) UnFurnished B...,Corporation water,"18.6259403,73.7804315","6,500","15,000",2021-03-10,Broker: Krishna estate agency
4,1 BHK Residential Apartment for Rent in Kothrud,/apartments-flats-for-rent/625-sqft-1-bhk-flat...,"Shivthirth Nagar, Kothrud, Pune View Map",No Car Parking 1st (of 3 Floors) UnFurnished F...,Corporation water Drainage Facility Security,"0,0","14,000","40,000",2021-03-10,Broker: Sachin Enterprises
5,2 BHK Residential Apartment for Rent in Magarp...,/apartments-flats-for-rent/930-sqft-2-bhk-apar...,"Magarpatta, Pune View Map",1 Car Parking 5th (of 11 Floors) UnFurnished F...,Lifts Park Club House +2 Amenities,"0,0","18,000","50,000",2021-02-23,Broker: Trimurti Real Estate consultant
6,1 RK Residential Apartment for Rent at Gurukru...,/apartments-flats-for-rent/400-sqft-1-rk-flat-...,"Kharadi, Pune View Map",No Car Parking 2nd (of 4 Floors) UnFurnished F...,Corporation water Drainage Facility,"0,0","6,000","15,000",2021-03-26,Building: Gurukrupa housing society Broker: ...
7,1 RK Residential Apartment for Rent at shree a...,/apartments-flats-for-rent/600-sqft-1-rk-flat-...,"Bhosari, Pimpri Chinchwad Landmark: centra...",1 Car Parking 4th (of 4 Floors) Semi Furnished...,Corporation water Drainage Facility Visitors P...,"18.6440005,73.8505065","8,000","20,000",2020-02-28,Building: shree apartment Broker: The Relia...
8,2 BHK Residential Apartment for Rent in Moshi ...,/apartments-flats-for-rent/860-sqft-2-bhk-apar...,"Moshi Pradhikaran, Pimpri Chinchwad View ...",1 Car Parking 3rd (of 4 Floors) UnFurnished Fa...,Corporation water Drainage Facility Visitors P...,"18.6527077,73.8480927",850,"30,000",2020-08-08,Broker: Prime Properties
9,1 BHK Residential Apartment for Rent at life R...,/apartments-flats-for-rent/680-sqft-1-bhk-flat...,"Marunji, Pimpri Chinchwad Landmark: life r...",1 Car Parking 19th (of 22 Floors) UnFurnished ...,Corporation water Drainage Facility 24*7 Power...,"18.6208973,73.7106885","13,000","40,000",2020-12-06,Building: life Republic Broker: Nimbleland...
